# 1. TF.Text
- text와 관련된 class 제공
- text-based model에서 필요한 preprocessing을 처리할 수 있도록 하는 library 제공

# 2. Eager Execution(즉시 실행)
- 그래프를 생성하지 않고 함수를 바로 실행하는 명령형 프로그래밍 환경으로 나중에 실행하기 위해 계산가능한 그래프를 생성하는 대신에 계산값을 즉시 알려주는 연산
- 연구와 실험을 위한 플랫폼

In [3]:
import tensorflow as tf
import tensorflow_text as text

# 3. Unicode
- UTF-8은 Unicode를 인코딩 하는 방식임
- String을 UTF-8 방식으로 바꿔주거나 되어있음

In [4]:
docs = tf.constant([u'Everything not saved will be lost.'.encode('UTF-16-BE'), u'Sad☹'.encode('UTF-16-BE')])
utf8_docs = tf.strings.unicode_transcode(docs, input_encoding='UTF-16-BE', output_encoding='UTF-8')

# 4. Tokenization
- string 을 tokens으로 나눠주는 것
- tokens는 단어(words), 숫자(numbers), 또는 punctuation을 말함

<b>`Tokenizer`</b>와 <b>`TokenizerWithOffsets`</b>은 각각 <b>`tokenize`</b>와 <b>`tokenize_with_offsets`</b> 메서드를 가짐

<b>`TokenizerWithOffsets`</b>은 byte offset을 original string으로 변환함

<b>`Tokenizer`</b>들은 <i>`RagggedTensors(비정형 텐서)`</i>를 반환함
  - 비정형 텐서는 가변길이의 데이터를 쉽게 저장하고 처리할 수 있음
  - 수학 연산 (예 : `tf.add` 및 `tf.reduce_mean`), 배열 연산 (예 : `tf.concat` 및 `tf.tile`), 문자열 조작 작업 (예 : `tf.substr`)을 포함하여 수백 가지 이상의 텐서플로 연산에서 지원
  - [비정형텐서 참고](https://www.tensorflow.org/guide/ragged_tensor)

### WhitespaceTokenizer
- UTF-8 string의 whitespace를 나눠줌(스페이스, 탭, 줄바꿈)

In [5]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.
[[b'everything', b'not', b'saved', b'will', b'be', b'lost.'], [b'Sad\xe2\x98\xb9']]


### UnicodeScriptTokenizer
- Unicode stript의 바운더리에 기반하여 UTF-8 string을 나눠줌(split)
- WhitespaceTokenizer와 거의 비슷한 역할을 하지만 추가로 구두점(punctuation, USCRIPT_COMMON)을 language text(USCRIPT_LATIN, USCRIPT_CYRILLIC 등)로 바꿔줄 수 있음

In [7]:
tokenizer = text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad🙁'.encode('UTF-8')])
print(tokens.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xf0\x9f\x99\x81']]


### Unicode split
- 언어(문장)을 whitespace 없이 단어로 토크나이징할 때 <i>unicode_split</i>을 사용해 철자(character)로 나눔(split)

In [8]:
tokens = tf.strings.unicode_split([u"仅今年前".encode('UTF-8')], 'UTF-8')
print(tokens.to_list())

[[b'\xe4\xbb\x85', b'\xe4\xbb\x8a', b'\xe5\xb9\xb4', b'\xe5\x89\x8d']]


### Offsets
- strings(문장)을 토크나이즈 할 때 original string이 어디서 왔는지 알아야 함
- 그래서 `TokenizerWithOffsets`을 implement하는 모든 `tokenizer`는 토큰과 함께 byte offset을 함께 리턴하는 `tokenize_with_offsets` 메서드를 가짐
- `offset_starts`는 각 token이 시작하는 original string의 byte를 리스트 함
- `offset_limits`는 각 토큰이 끝나는 byte를 리스트 함

In [9]:
tokenizer = text.UnicodeScriptTokenizer()
(tokens, offset_starts, offset_limits) = tokenizer.tokenize_with_offsets(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())
print(offset_starts.to_list())
print(offset_limits.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]
[[0, 11, 15, 21, 26, 29, 33], [0, 3]]
[[10, 14, 20, 25, 28, 33, 34], [3, 6]]


### TF.Data Example
- tokenizer는 tf.data.API 가 기대하는데로 작동함

In [10]:
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: cannot import name 'DenseNet121' from 'tensorflow.python.keras.applications' (/usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/applications/__init__.py)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: cannot import name 'DenseNet121' from 'tensorflow.python.keras.applications' (/usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/applications/__init__.py)
[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]


# 5. Other Text ops
- `tf.text` 는 다른 전처리 기능을 포함함

### 1) Wordshape
- NLP model에서 고려하는 feature는 text string이 특정한 특징을 가지고 있는지 분석하는 것임.
- 예를 들어 이모티콘이 있거나 대문자로 쓰여진 단어를 확인함

In [11]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Is capitalized?
f1 = text.wordshape(tokens, text.WordShape.HAS_TITLE_CASE)
# Are all letters uppercased?
f2 = text.wordshape(tokens, text.WordShape.IS_UPPERCASE)
# Does the token contain punctuation?
f3 = text.wordshape(tokens, text.WordShape.HAS_SOME_PUNCT_OR_SYMBOL)
# Is the token a number?
f4 = text.wordshape(tokens, text.WordShape.IS_NUMERIC_VALUE)

print(f1.to_list())
print(f2.to_list())
print(f3.to_list())
print(f4.to_list())

[[True, False, False, False, False, False], [True]]
[[False, False, False, False, False, False], [False]]
[[False, False, False, False, False, True], [True]]
[[False, False, False, False, False, False], [False]]


### 2) N-grams & Sliding Window
- 단어의 연속 with <i>n</i> 크기를 가지는 sliding window
- `Reduction.STRING_JOIN`을 통해 string을 각각에 붙여 tokens를 축소를 할 수 있음
- 디폴트는 띄어쓰기이며 `string_separater`로 조정 가능

In [12]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Ngrams, in this case bi-gram (n = 2)
bigrams = text.ngrams(tokens, 2, reduction_type=text.Reduction.STRING_JOIN)

print(bigrams.to_list())

[[b'Everything not', b'not saved', b'saved will', b'will be', b'be lost.'], []]
